![Title](https://i.imgur.com/TD4tCAo.png)

In [1]:
import pandas as pd
import numpy as np

In [2]:
iy = pd.read_csv('../dataset/dmtg.csv')

In [3]:
iy.columns

Index(['Unnamed: 0', 'Encounter', 'ID', 'Facility Name', 'Bus St Date',
       'Description', 'Inspect Dt_x', 'Low_x', 'Medium_x', 'High_x',
       'business_id', 'address', 'price', 'cuisine_Asian Fusion',
       'cuisine_Vegetarian', 'cuisine_Chinese',
       'cuisine_American (Traditional)', 'cuisine_Italian', 'cuisine_Mexican',
       'cuisine_Thai', 'cuisine_others', 'cuisine_American (New)',
       'cuisine_Indian', 'cuisine_Japanese', 'cuisine_Greek',
       'cuisine_Mediterranean', 'hyg_0', 'hyg_1', 'hyg_2', 'hyg_3', 'hyg_4',
       'hyg_5', 'unh_0', 'unh_1', 'unh_2', 'unh_3', 'stars_y', 'count',
       'Low_y', 'Medium_y', 'High_y'],
      dtype='object')

In [4]:
iy.loc[iy['High_x']>0, 'isHigh'] = 1
iy['isHigh'] = iy['isHigh'].fillna(0)

In [5]:
iy['ID'].unique().shape[0] # In the previous version, this number is 1243

805

In [6]:
iy['isHigh'].value_counts()

1.0    1293
0.0    1010
Name: isHigh, dtype: int64

In [7]:
# Features and target variable
features = ['price', # price level
            'cuisine_Asian Fusion', 'cuisine_Vegetarian', 'cuisine_Chinese', # cuisine
            'cuisine_American (Traditional)', 'cuisine_Italian', 'cuisine_Mexican',
            'cuisine_Thai', 'cuisine_others', 'cuisine_American (New)', 'cuisine_Indian',
            'cuisine_Japanese','cuisine_Greek', 'cuisine_Mediterranean', 
            'hyg_0', 'hyg_1', 'hyg_2','hyg_3', 'hyg_4', 'hyg_5', # dictionary words
            'unh_0', 'unh_1', 'unh_2', 'unh_3',
            'stars_y', 'count', # average star ratings and review count
            'Low_y', 'Medium_y', 'High_y'] # 
target = 'isHigh'

In [8]:
# Check columns with missing values
pd.isnull(iy).sum()[pd.isnull(iy).sum()>0]

Series([], dtype: int64)

In [9]:
# Imputation
## Imputation is not needed because there is no missig value in the dataset

In [10]:
# Preprocessing and Evaluation
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler 
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, roc_curve, auc

# Model
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

# split the dataset into predictors(X) and target (Y)
X = iy[features]
Y = iy[target].values.ravel()

## 0) Data split for Cross-validation

In [11]:
# splitting into three sets
X_trainval, X_test, Y_trainval, Y_test = train_test_split(
    X, Y, random_state=0)

# Feature scaling
scaler = MinMaxScaler().fit(X_trainval)
X_trainval_scaled = scaler.transform(X_trainval)
X_test_scaled = scaler.transform(X_test)

# Generate report
def getReport(report):
    report = report.strip().split()
    report.remove('avg')
    report.remove('/')
    report.insert(0, 'class')
    report = np.array(report)
    report = np.reshape(report, [-1,5])
    colname = report[0,:]
    report = report[1:,:]
    return pd.DataFrame(report, columns=colname)

In [12]:
acc = [] # list to store all performance metric

## A) Logistic Regression
The parameter C, inverse of regularization strength.

Tuning range: [0.001, 0.1, 1, 10, 100]

In [13]:
# Dataset with imputation
best_score=0
kfolds=5 # set the number of folds

for c in [0.001, 0.1, 1, 10, 100]:
    logRegModel = LogisticRegression(C=c)
    # perform cross-validation
    scores = cross_val_score(logRegModel, X_trainval_scaled, Y_trainval, cv=kfolds, scoring='accuracy') # Get recall for each parameter setting
    
    # compute mean cross-validation accuracy
    score = np.mean(scores)
    
    # Find the best parameters and score
    if score > best_score:
        best_score = score
        best_parameters = c

# rebuild a model on the combined training and validation set
SelectedLogRegModel = LogisticRegression(C=best_parameters).fit(X_trainval_scaled, Y_trainval)

test_score = SelectedLogRegModel.score(X_test_scaled, Y_test)
PredictedOutput = SelectedLogRegModel.predict(X_test_scaled)
test_recall = recall_score(Y_test, PredictedOutput, pos_label=1)
fpr, tpr, thresholds = roc_curve(Y_test, PredictedOutput, pos_label=1)
test_auc = auc(fpr, tpr)
print("Best accuracy on validation set is:", best_score)
print("Best parameter for regularization (C) is: ", best_parameters)
print("Test accuracy with best C parameter is", test_score)
print("Test recall with the best C parameter is", test_recall)
print("Test AUC with the best C parameter is", test_auc)
m = 'Logistic Regression'
acc.append([m, test_score, test_recall, test_auc, fpr, tpr, thresholds])

Best accuracy on validation set is: 0.695976181153
Best parameter for regularization (C) is:  1
Test accuracy with best C parameter is 0.664930555556
Test recall with the best C parameter is 0.687878787879
Test AUC with the best C parameter is 0.661012564671


In [14]:
pd.DataFrame([features, SelectedLogRegModel.coef_.tolist()[0]]).T

,0,1
0,price,0.231729
1,cuisine_Asian Fusion,0.0507491
2,cuisine_Vegetarian,-0.163093
3,cuisine_Chinese,0.0839621
4,cuisine_American (Traditional),0.255647
5,cuisine_Italian,-0.188375
6,cuisine_Mexican,-0.263167
7,cuisine_Thai,0.15934
8,cuisine_others,0.0538622
9,cuisine_American (New),0.465049


In [15]:
X_test_df = pd.concat([pd.DataFrame(X_test, columns=features), pd.Series(iy['Facility Name'])], axis=1, join='inner')
output = pd.concat([X_test_df.reset_index(drop=True), pd.DataFrame(Y_test, columns=['True']), pd.DataFrame(PredictedOutput, columns=['Prediction'])], axis=1)
output = output.rename(columns={0: 'True', 1: 'Prediction'})
output.to_csv('../test_output_log_reg.csv')

## B) Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

# your GNB estimator
gnb = GaussianNB()
gnb.fit(X_trainval_scaled, Y_trainval)

test_score = gnb.score(X_test_scaled, Y_test)
PredictedOutput = gnb.predict(X_test_scaled)
test_recall = recall_score(Y_test, PredictedOutput, pos_label=1)
fpr, tpr, thresholds = roc_curve(Y_test, PredictedOutput, pos_label=1)
test_auc = auc(fpr, tpr)
print("Test accuracy with the best parameters is", test_score)
print("Test recall with the best parameters is", test_recall)
print("Test recall with the best parameter is", test_auc)

m = 'GNB'
acc.append([m, test_score, test_recall, test_auc, fpr, tpr, thresholds])

## C) SVM
C: Penalty parameter C of the error term.

gamma: kernel coefficient. 

kernel: kernel type. 

In [16]:
best_score = 0

for c_paramter in [0.001, 0.1, 10, 100]: #iterate over the values we need to try for the parameter C
    for gamma_paramter in [0.001, 0.1, 10, 100]: #iterate over the values we need to try for the parameter gamma
        for k_parameter in ['rbf']: # iterate over the values we need to try for the kernel parameter
            print(c_paramter, gamma_paramter, k_parameter)
            svmModel = SVC(kernel=k_parameter, C=c_paramter, gamma=gamma_paramter) #define the model
            # perform cross-validation
            scores = cross_val_score(svmModel, X_trainval_scaled, Y_trainval, cv=kfolds, scoring='accuracy')
            # the training set will be split internally into training and cross validation

            # compute mean cross-validation accuracy
            score = np.mean(scores)
            # if we got a better score, store the score and parameters
            if score > best_score:
                best_score = score #store the score 
                best_parameter_c = c_paramter #store the parameter c
                best_parameter_gamma = gamma_paramter #store the parameter gamma
                best_parameter_k = k_parameter
            

# rebuild a model with best parameters to get score 
SelectedSVMmodel = SVC(C=best_parameter_c, gamma=best_parameter_gamma, kernel=best_parameter_k).fit(X_trainval_scaled, Y_trainval)

test_score = SelectedSVMmodel.score(X_test_scaled, Y_test)
PredictedOutput = SelectedSVMmodel.predict(X_test_scaled)
test_recall = recall_score(Y_test, PredictedOutput, pos_label=1)
fpr, tpr, thresholds = roc_curve(Y_test, PredictedOutput, pos_label=1)
test_auc = auc(fpr, tpr)
print("Best accuracy on cross validation set is:", best_score)
print("Best parameter for c is: ", best_parameter_c)
print("Best parameter for gamma is: ", best_parameter_gamma)
print("Best parameter for kernel is: ", best_parameter_k)
print("Test accuracy with the best parameters is", test_score)
print("Test recall with the best parameters is", test_recall)
print("Test recall with the best parameter is", test_auc)

m = 'SVM'
acc.append([m, test_score, test_recall, test_auc, fpr, tpr, thresholds])

0.001 0.001 rbf
0.001 0.1 rbf
0.001 10 rbf
0.001 100 rbf
0.1 0.001 rbf
0.1 0.1 rbf
0.1 10 rbf
0.1 100 rbf
10 0.001 rbf
10 0.1 rbf
10 10 rbf
10 100 rbf
100 0.001 rbf
100 0.1 rbf
100 10 rbf
100 100 rbf
Best accuracy on cross validation set is: 0.702932722374
Best parameter for c is:  10
Best parameter for gamma is:  0.1
Best parameter for kernel is:  rbf
Test accuracy with the best parameters is 0.668402777778
Test recall with the best parameters is 0.715151515152
Test recall with the best parameter is 0.660421286031


## D) Decision Tree
Maximum depth. [1, 2, ..., 5]


In [17]:
best_score = 0

for md in range(1, 6): # iterate different maximum depth values
    # train the model
    treeModel = DecisionTreeClassifier(random_state=0, max_depth=md, criterion='gini')
    # perform cross-validation
    scores = cross_val_score(treeModel, X_trainval_scaled, Y_trainval, cv=kfolds, scoring='accuracy')
    
    # compute mean cross-validation accuracy
    score = np.mean(scores)
    
    # if we got a better score, store the score and parameters
    if score > best_score:
        best_score = score
        best_parameter = md

# Rebuild a model on the combined training and validation set        
SelectedDTModel = DecisionTreeClassifier(max_depth=best_parameter).fit(X_trainval_scaled, Y_trainval )

test_score = SelectedDTModel.score(X_test_scaled, Y_test)
PredictedOutput = SelectedDTModel.predict(X_test_scaled)
test_recall = recall_score(Y_test, PredictedOutput, pos_label=1)
fpr, tpr, thresholds = roc_curve(Y_test, PredictedOutput, pos_label=1)
test_auc = auc(fpr, tpr)
print("Best accuracy on validation set is:", best_score)
print("Best parameter for the maximum depth is: ", best_parameter)
print("Test accuracy with best parameter is ", test_score)
print("Test recall with best parameters is ", test_recall)
print("Test AUC with the best parameter is ", test_auc)

m = 'Decision Tree'
acc.append([m, test_score, test_recall, test_auc, fpr, tpr, thresholds])

Best accuracy on validation set is: 0.69944938641
Best parameter for the maximum depth is:  2
Test accuracy with best parameter is  0.661458333333
Test recall with best parameters is  0.821212121212
Test AUC with the best parameter is  0.634183296378


## E) Random Forest Classifier
n_estimators(M): the number of trees in the forest

max_features(d): the number of features to consider when looking for the best split

max_depth(m): the maximum depth of the tree. 

In [18]:
best_score = 0

for M in range(2, 50, 1): # combines M trees
    for d in range(1, 13): # maximum number of features considered at each split
        for m in range(1, 9): # maximum depth of the tree
            # train the model
            # n_jobs(-1) is the maximum number of parallel computing
            forestModel = RandomForestClassifier(n_estimators=M, max_features=d, n_jobs=-1,
                                          max_depth=m, random_state=0)
        
            # perform cross-validation
            scores = cross_val_score(forestModel, X_trainval_scaled, Y_trainval, cv=kfolds, scoring='accuracy')

            # compute mean cross-validation accuracy
            score = np.mean(scores)

            # if we got a better score, store the score and parameters
            if score > best_score:
                best_score = score
                best_M = M
                best_d = d
                best_m = m

# Rebuild a model on the combined training and validation set        
SelectedRFModel = RandomForestClassifier(n_estimators=M, max_features=d,
                                          max_depth=m, random_state=0).fit(X_trainval_scaled, Y_trainval )

PredictedOutput = SelectedRFModel.predict(X_test_scaled)
test_score = SelectedRFModel.score(X_test_scaled, Y_test)
test_recall = recall_score(Y_test, PredictedOutput, pos_label=1)
fpr, tpr, thresholds = roc_curve(Y_test, PredictedOutput, pos_label=1)
test_auc = auc(fpr, tpr)
print("Best accuracy on validation set is:", best_score)
print("Best parameters of M, d, m are: ", best_M, best_d, best_m)
print("Test accuracy with the best parameters is", test_score)
print("Test recall with the best parameters is:", test_recall)
print("Test AUC with the best parameters is:", test_auc)

m = 'Random Forest'
acc.append([m, test_score, test_recall, test_auc, fpr, tpr, thresholds])

Best accuracy on validation set is: 0.717400363536
Best parameters of M, d, m are:  34 6 2
Test accuracy with the best parameters is 0.663194444444
Test recall with the best parameters is: 0.739393939394
Test AUC with the best parameters is: 0.650184774575


In [19]:
pd.DataFrame([features, SelectedRFModel.feature_importances_.tolist()]).T

,0,1
0,price,0.0105347
1,cuisine_Asian Fusion,0.000937499
2,cuisine_Vegetarian,0.00180275
3,cuisine_Chinese,0.00251821
4,cuisine_American (Traditional),0.00548718
5,cuisine_Italian,0.0121878
6,cuisine_Mexican,0.00431027
7,cuisine_Thai,0.00388832
8,cuisine_others,0.00386718
9,cuisine_American (New),0.00788052


## F) AdaBoost

In [20]:
best_score = 0

for M in range(2, 30, 2): # combines M trees
    for lr in [0.001, 0.01, 0.1, 1]:
        # train the model
        boostModel = AdaBoostClassifier(n_estimators=M, learning_rate=lr, random_state=0)

        # perform cross-validation
        scores = cross_val_score(boostModel, X_trainval_scaled, Y_trainval, cv=kfolds, scoring='accuracy')

        # compute mean cross-validation accuracy
        score = np.mean(scores)

        # if we got a better score, store the score and parameters
        if score > best_score:
            best_score = score
            best_M = M
            best_lr = lr

# Rebuild a model on the combined training and validation set        
SelectedBoostModel = AdaBoostClassifier(n_estimators=M, learning_rate=lr, random_state=0).fit(X_trainval_scaled, Y_trainval )

PredictedOutput = SelectedBoostModel.predict(X_test_scaled)
test_score = SelectedRFModel.score(X_test_scaled, Y_test)
test_recall = recall_score(Y_test, PredictedOutput, pos_label=1)
fpr, tpr, thresholds = roc_curve(Y_test, PredictedOutput, pos_label=1)
test_auc = auc(fpr, tpr)
print("Best accuracy on validation set is:", best_score)
print("Best parameter of M is: ", best_M)
print("best parameter of LR is: ", best_lr)
print("Test accuracy with the best parameter is", test_score)
print("Test recall with the best parameters is:", test_recall)
print("Test AUC with the best parameters is:", test_auc)

m = 'AdaBoost'
acc.append([m, test_score, test_recall, test_auc, fpr, tpr, thresholds])

Best accuracy on validation set is: 0.702937748762
Best parameter of M is:  28
best parameter of LR is:  0.1
Test accuracy with the best parameter is 0.663194444444
Test recall with the best parameters is: 0.715151515152
Test AUC with the best parameters is: 0.658388765706


## Results

In [21]:
result = pd.DataFrame(acc, columns=['Model', 'Accuracy', 'Recall', 'AUC', 'TPR', 'FPR', 'TH'])
result[['Model', 'Accuracy', 'Recall', 'AUC']]

,Model,Accuracy,Recall,AUC
0,Logistic Regression,0.664931,0.687879,0.661013
1,SVM,0.668403,0.715152,0.660421
2,Decision Tree,0.661458,0.821212,0.634183
3,Random Forest,0.663194,0.739394,0.650185
4,AdaBoost,0.663194,0.715152,0.658389


In [22]:
# Previous result
'''	Model	Accuracy	Recall	AUC
0	Logistic Regression	0.601050	0.391137	0.583389
1	SVM	0.570429	0.337187	0.550805
2	Decision Tree	0.570429	0.169557	0.536701
3	Random Forest	0.570429	0.366089	0.553237
4	AdaBoost	0.570429	0.431599	0.581184 '''

'\tModel\tAccuracy\tRecall\tAUC\n0\tLogistic Regression\t0.601050\t0.391137\t0.583389\n1\tSVM\t0.570429\t0.337187\t0.550805\n2\tDecision Tree\t0.570429\t0.169557\t0.536701\n3\tRandom Forest\t0.570429\t0.366089\t0.553237\n4\tAdaBoost\t0.570429\t0.431599\t0.581184 '

## Comment
This model can be improved with several ways.

- Dimension reduction
- Using only unhygine dictionary words (Feature Selection)
- Applying word2vec to Yelp review and tips data